# 06 - Model Risk: Stochastic Volatility and Smile

Generate a smile from a Heston-lite world and compare hedging errors against the Black-Scholes world.

In [4]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
!ls /content/interactive_portfolio_optimization


/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 83 (delta 33), reused 42 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 54.03 KiB | 9.00 MiB/s, done.
Resolving deltas: 100% (33/33), done.
data  notebooks  pyproject.toml  README.md  results  src  tests


In [5]:
from pathlib import Path
print((Path("/content/interactive_portfolio_optimization/src")).exists())
print((Path("/content/interactive_portfolio_optimization/pyproject.toml")).exists())


True
True


In [6]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if not (ROOT / "src").exists():
    raise FileNotFoundError(f"Bad ROOT: {ROOT}")

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT OK:", ROOT)
print("tables exists:", (ROOT / "results" / "tables").exists())


ROOT OK: /content/interactive_portfolio_optimization
tables exists: True


In [7]:
import numpy as np
import pandas as pd

from src.config import config_dict
from src.stoch_vol import simulate_heston_lite_paths, sv_option_prices_mc, implied_vol_smile_from_sv_prices
from src.hedging import simulate_delta_hedge_on_paths, simulate_delta_hedge_gbm
from src.plotting import plot_smile

cfg = config_dict(fast_mode=True)
print("imports + cfg ready")


imports + cfg ready


In [8]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import numpy as np
import pandas as pd

from src.config import config_dict
from src.stoch_vol import simulate_heston_lite_paths, sv_option_prices_mc, implied_vol_smile_from_sv_prices
from src.hedging import simulate_delta_hedge_on_paths, simulate_delta_hedge_gbm
from src.plotting import plot_smile

cfg = config_dict(fast_mode=True)


In [10]:
from pathlib import Path

print("ROOT =", ROOT)
print("exists ROOT:", Path(ROOT).exists())

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

print("tables exists:", (ROOT/"results"/"tables").exists())
print("figures exists:", (ROOT/"results"/"figures").exists())


ROOT = /
exists ROOT: True
tables exists: True
figures exists: True


In [11]:
strike_grid = [80, 90, 100, 110, 120]
maturities = [0.5, 1.0]
price_table = sv_option_prices_mc(
    strike_grid=strike_grid,
    maturities=maturities,
    S0=cfg['S0'],
    r=cfg['R'],
    q=cfg['Q'],
    V0=cfg['V0'],
    kappa=cfg['KAPPA'],
    theta=cfg['THETA'],
    xi=cfg['XI'],
    rho=cfg['RHO'],
    n_paths=40_000,
    n_steps_per_year=252,
    option_type='call',
    seed=cfg['SEED'],
)
smile = implied_vol_smile_from_sv_prices(price_table, S=cfg['S0'], r=cfg['R'], q=cfg['Q'])
smile.to_csv(ROOT / 'results' / 'tables' / 'sv_smile.csv', index=False)
plot_smile(smile, ROOT / 'results' / 'figures' / 'sv_smile.png')
smile.head()


,maturity,strike,option_type,price,implied_vol
0,0.5,80,call,21.572813,0.258407
1,0.5,90,call,12.877191,0.221521
2,0.5,100,call,5.829652,0.189580
3,0.5,110,call,1.675476,0.166220
4,0.5,120,call,0.314460,0.157035


In [12]:
atm_row = smile[(smile['maturity'] == 1.0) & (smile['strike'] == cfg['K'])]
if atm_row.empty:
    sigma_hedge = float(smile['implied_vol'].dropna().median())
else:
    sigma_hedge = float(atm_row['implied_vol'].iloc[0])

sv_paths = simulate_heston_lite_paths(
    S0=cfg['S0'], r=cfg['R'], q=cfg['Q'], V0=cfg['V0'], kappa=cfg['KAPPA'], theta=cfg['THETA'],
    xi=cfg['XI'], rho=cfg['RHO'], T=1.0, n_paths=6_000, n_steps=252, seed=cfg['SEED']
)
sv_hedge = simulate_delta_hedge_on_paths(
    paths=sv_paths, K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma_model=sigma_hedge,
    T=1.0, rebalance_every_k_steps=5, option_type='call', tx_cost_per_dollar=0.0
)
bs_hedge = simulate_delta_hedge_gbm(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=sigma_hedge, T=1.0,
    n_paths=6_000, n_steps=252, rebalance_every_k_steps=5, option_type='call',
    tx_cost_per_dollar=0.0, seed=cfg['SEED']
)

model_risk = pd.DataFrame([
    {'world': 'BS', 'mean_error': bs_hedge['mean_error'], 'std_error': bs_hedge['std_error'], 'q05': bs_hedge['q05'], 'q95': bs_hedge['q95']},
    {'world': 'SV_misspecified_delta', 'mean_error': sv_hedge['mean_error'], 'std_error': sv_hedge['std_error'], 'q05': sv_hedge['q05'], 'q95': sv_hedge['q95']},
])
model_risk.to_csv(ROOT / 'results' / 'tables' / 'model_risk_hedge_comparison.csv', index=False)
model_risk


,world,mean_error,std_error,q05,q95
0,BS,0.00029,0.897494,-1.494703,1.445597
1,SV_misspecified_delta,0.00777,2.763332,-4.953381,3.588430


Notebook 06 Summary

We moved from the Black–Scholes constant-volatility framework to a stochastic-volatility setting (Heston-type), where variance evolves over time and is correlated with asset returns.

Model:


The joint dynamics are given by

$dV_t = \kappa(\theta - V_t)\,dt + \xi\sqrt{V_t}\,dW_t^{(2)}$

$dS_t = (r - q)\,S_t\,dt + \sqrt{V_t}\,S_t,dW_t^{(1)}$

with $\mathrm{corr}\!\left(W_t^{(1)}, W_t^{(2)}\right)=\rho.$



Implied volatility extraction:

From simulated option prices under stochastic volatility, implied volatilities were obtained by solving

$C_{\mathrm{BS}}(S,K,r,q,\sigma_{\mathrm{imp}},T) = C_{\mathrm{SV}}$

Smile results (for $T=0.5$):

•	$K=80$: $\sigma_{\mathrm{imp}} \approx 0.258$


•	$K=100$: $\sigma_{\mathrm{imp}} \approx 0.190$


•	$K=120$: $\sigma_{\mathrm{imp}} \approx 0.157$

Implied volatility is clearly not flat and decreases with strike, indicating a pronounced skew relative to Black–Scholes assumptions.

Hedging comparison (model risk):

•	Black–Scholes hedge: standard deviation $\approx 0.897$, $q_{0.05} \approx -1.495$.

•	Stochastic-volatility world with BS delta (misspecified hedge): standard deviation $\approx 2.763$, $q_{0.05} \approx -4.953$.

Findings:

•	Mean hedging errors are close to zero in both cases, indicating little average bias.

•	Risk dispersion increases sharply under volatility misspecification (roughly $3\times$ higher standard deviation).

•	Left-tail losses deteriorate substantially when using a constant-vol hedge in a stochastic-vol environment.


Key takeaway:

Even when average hedging P&L appears acceptable, model misspecification can severely amplify tail risk.
Constant-volatility hedges underestimate risk under stochastic volatility, highlighting that effective risk management must focus on distributional and tail outcomes, not only mean errors.